In [1]:
'''
    @Author: King
    @Date: 2019.06.25
    @Purpose: Graph Convolutional Network
    @Introduction:   This is a gentle introduction of using DGL to implement 
                    Graph Convolutional Networks (Kipf & Welling et al., 
                    Semi-Supervised Classification with Graph Convolutional Networks). 
                    We build upon the earlier tutorial on DGLGraph and demonstrate how DGL 
                    combines graph with deep neural network and learn structural representations.
    @Datasets: 
    @Link : 
    @Reference : https://docs.dgl.ai/tutorials/models/1_gnn/1_gcn.html
'''

'\n    @Author: King\n    @Date: 2019.06.25\n    @Purpose: Graph Convolutional Network\n    @Introduction:   \n    @Datasets: \n    @Link : \n    @Reference : https://docs.dgl.ai/tutorials/models/1_gnn/1_gcn.html\n'

# Graph Convolutional Network

## Model Overview

### GCN from the perspective of message passing

We describe a layer of graph convolutional neural network from a message passing perspective; the math can be found here(https://docs.dgl.ai/tutorials/models/1_gnn/1_gcn.html#math). It boils down to the following step, for each node u:


1) Aggregate neighbors’ representations $h_{v}$ to produce an intermediate representation $\hat{h}_{u}$.
2) Transform the aggregated representation $\hat{h}_{u}$ with a linear projection followed by a non-linearity: $h_{u}=f\left(W_{u} \hat{h}_{u}\right)$.

We will implement step 1 with DGL message passing, and step 2 with the apply_nodes method, whose node UDF will be a PyTorch nn.Module.


### GCN implementation with DGL

We first define the message and reduce function as usual. Since the aggregation on a node u only involves summing over the neighbors’ representations $h_{v}$, 
(我们像往常一样定义message and reduce function。由于节点uu上的聚合仅涉及对邻居的表示 $h_{v}$ 求和)

we can simply use builtin functions:

In [2]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

We then define the node UDF for apply_nodes, which is a fully-connected layer:

In [3]:
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

We then proceed to define the GCN module. A GCN layer essentially performs message passing on all the nodes then applies the NodeApplyModule. Note that we omitted the dropout in the paper for simplicity.

然后我们继续定义GCN模块。 GCN层实质上在所有节点上执行消息传递，然后应用NodeApplyModule。 请注意，为简单起见，我们省略了文中的丢失。

In [4]:
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(gcn_msg, gcn_reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

The forward function is essentially the same as any other commonly seen NNs model in PyTorch. 

We can initialize GCN like any nn.Module. For example, let’s define a simple neural network consisting of two GCN layers. Suppose we are training the classifier for the cora dataset (the input feature size is 1433 and the number of classes is 7).

我们可以像任何nn.Module一样初始化GCN。例如，让我们定义一个由两个GCN层组成的简单神经网络。假设我们正在训练cora数据集的分类器（输入要素大小为1433，类的数量为7）。

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCN(1433, 16, F.relu)
        self.gcn2 = GCN(16, 7, F.relu)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.gcn2(g, x)
        return x
net = Net()
print(net)

Net(
  (gcn1): GCN(
    (apply_mod): NodeApplyModule(
      (linear): Linear(in_features=1433, out_features=16, bias=True)
    )
  )
  (gcn2): GCN(
    (apply_mod): NodeApplyModule(
      (linear): Linear(in_features=16, out_features=7, bias=True)
    )
  )
)


We load the cora dataset using DGL’s built-in data module.

In [6]:
from dgl.data import citation_graph as citegrh
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    mask = th.ByteTensor(data.train_mask)
    g = data.graph
    # add self loop
    g.remove_edges_from(g.selfloop_edges())
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    return g, features, labels, mask

We then train the network as follows:

In [7]:
import time
import numpy as np
g, features, labels, mask = load_cora_data()
optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
dur = []
for epoch in range(30):
    if epoch >=3:
        t0 = time.time()

    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), np.mean(dur)))

Extracting file to C:\Users\ASUS\.dgl/cora


D:\progrom\python\python\python3\lib\site-packages\dgl\base.py:18: UserWarning: Initializer is not set. Use zero initializer instead. To suppress this warning, use `set_initializer` to explicitly specify which initializer to use.
  warnings.warn(msg)
D:\progrom\python\python\python3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\progrom\python\python\python3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.9448 | Time(s) nan
Epoch 00001 | Loss 1.9262 | Time(s) nan
Epoch 00002 | Loss 1.9091 | Time(s) nan
Epoch 00003 | Loss 1.8922 | Time(s) 0.3211
Epoch 00004 | Loss 1.8756 | Time(s) 0.3181
Epoch 00005 | Loss 1.8603 | Time(s) 0.3191
Epoch 00006 | Loss 1.8452 | Time(s) 0.3221
Epoch 00007 | Loss 1.8301 | Time(s) 0.3209
Epoch 00008 | Loss 1.8152 | Time(s) 0.3211
Epoch 00009 | Loss 1.8005 | Time(s) 0.3253
Epoch 00010 | Loss 1.7861 | Time(s) 0.3236
Epoch 00011 | Loss 1.7720 | Time(s) 0.3224
Epoch 00012 | Loss 1.7585 | Time(s) 0.3214
Epoch 00013 | Loss 1.7454 | Time(s) 0.3218
Epoch 00014 | Loss 1.7324 | Time(s) 0.3231
Epoch 00015 | Loss 1.7196 | Time(s) 0.3230
Epoch 00016 | Loss 1.7070 | Time(s) 0.3219
Epoch 00017 | Loss 1.6946 | Time(s) 0.3234
Epoch 00018 | Loss 1.6826 | Time(s) 0.3228
Epoch 00019 | Loss 1.6709 | Time(s) 0.3250
Epoch 00020 | Loss 1.6596 | Time(s) 0.3260
Epoch 00021 | Loss 1.6485 | Time(s) 0.3272
Epoch 00022 | Loss 1.6377 | Time(s) 0.3301
Epoch 00023 | Loss 1

### GCN in one formula

Mathematically, the GCN model follows this formula:

$$
H^{(l+1)}=\sigma\left(\tilde{D}^{-\frac{1}{2}} \tilde{A} \tilde{D}^{-\frac{1}{2}} H^{(l)} W^{(l)}\right)
$$